# Kidney Disease Classification — Kaggle Training (4-Class)

This notebook runs the full pipeline on **Kaggle GPU** for the 4-class CT kidney dataset (Cyst, Normal, Stone, Tumor).

**Prerequisites:** Enable **GPU** (Settings → Accelerator → GPU T4x2/P100) and **Internet** (Settings → Internet → On).

In [ ]:
import os, sys
import tensorflow as tf
print('Python:', sys.version)
print('TensorFlow:', tf.__version__)
print('GPU devices:', tf.config.list_physical_devices('GPU'))

## 1) Clone repository

In [ ]:
%cd /kaggle/working
!rm -rf Kidney-Disease-Classification-System
!git clone https://github.com/saksham-1304/Kidney-Disease-Classification-System.git
%cd /kaggle/working/Kidney-Disease-Classification-System
!git pull

## 2) Install dependencies

Kaggle already has TensorFlow and the Kaggle CLI. We install the remaining packages.

In [ ]:
!pip install -q --upgrade pip

# Filter out tensorflow, mlflow, kaggle (Kaggle already has TF and kaggle CLI)
import re
with open('requirements.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
skip = re.compile(r'^\s*(tensorflow|mlflow|kaggle)\b', re.IGNORECASE)
filtered = [ln for ln in lines if not skip.match(ln)]
with open('requirements_kaggle.txt', 'w', encoding='utf-8') as f:
    f.writelines(filtered)

!pip install -q -r requirements_kaggle.txt
!pip install -q "mlflow>=2.16,<3.0"
!pip install -q ensure==1.0.2

# Install the project package
!pip install -q -e .

## 3) Verify Kaggle API access

On Kaggle notebooks the API is pre-authenticated — no `kaggle.json` upload needed. Just verify it works:

In [ ]:
!kaggle datasets list -s "ct-kidney" | head -n 5

## 4) Run pipeline stages

Set `START_STAGE` based on where you want to begin:
- `1`: full pipeline (data ingestion + base model + training + evaluation)
- `3`: training + evaluation only (if stage 1/2 artifacts already exist)

In [ ]:
import os
import subprocess
import sys
from collections import deque

START_STAGE = 1  # change to 3 if you already have stage 1/2 artifacts

stages = [
    (1, 'cnnClassifier.pipeline.stage_01_data_ingestion'),
    (2, 'cnnClassifier.pipeline.stage_02_prepare_base_model'),
    (3, 'cnnClassifier.pipeline.stage_03_model_training'),
    (4, 'cnnClassifier.pipeline.stage_04_model_evaluation'),
]

def run_stage(stage_num, module_name):
    print(f'\n===== Running stage {stage_num}: {module_name} =====')
    cmd = [sys.executable, '-m', module_name]
    tail = deque(maxlen=200)

    with subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    ) as proc:
        for line in proc.stdout:
            print(line, end='')
            tail.append(line.rstrip('\n'))

        return_code = proc.wait()

    if return_code != 0:
        print('\n----- LAST 200 LOG LINES -----')
        print('\n'.join(tail))
        raise RuntimeError(f'Stage {stage_num} failed with exit code {return_code}')

for stage_num, module_name in stages:
    if START_STAGE <= stage_num:
        run_stage(stage_num, module_name)

# Copy final model for serving
os.makedirs('model', exist_ok=True)
if os.path.exists('artifacts/training/model.h5'):
    import shutil
    shutil.copy2('artifacts/training/model.h5', 'model/model.h5')
    print('\nPipeline complete. Final model at model/model.h5')
else:
    print('\nPipeline did not produce artifacts/training/model.h5. Check logs above.')

## 5) Inspect evaluation results

In [ ]:
import json

if os.path.exists('scores.json'):
    with open('scores.json') as f:
        scores = json.load(f)
    print(f"K-Fold Cross-Validation Results (k={scores.get('k_folds', '?')}):")
    print(f"  Accuracy:        {scores.get('mean_accuracy', '?')} +/- {scores.get('std_accuracy', '?')}")
    print(f"  Loss:            {scores.get('mean_loss', '?')} +/- {scores.get('std_loss', '?')}")
    print(f"  Macro F1:        {scores.get('mean_macro_f1', '?')} +/- {scores.get('std_macro_f1', '?')}")
    print(f"  Macro Precision: {scores.get('mean_macro_precision', '?')} +/- {scores.get('std_macro_precision', '?')}")
    print(f"  Macro Recall:    {scores.get('mean_macro_recall', '?')} +/- {scores.get('std_macro_recall', '?')}")
    print(f"  Total samples:   {scores.get('total_samples', '?')}")
else:
    print('scores.json not found — evaluation stage may not have run.')

## 6) Download trained outputs

Run this cell to zip and download the trained model and scores. On Kaggle, the output will also appear in the Output tab of the notebook.

In [ ]:
import shutil

# Collect outputs
output_dir = '/kaggle/working/trained_outputs'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'model'), exist_ok=True)

# Copy final model
if os.path.exists('model/model.h5'):
    shutil.copy2('model/model.h5', os.path.join(output_dir, 'model', 'model.h5'))

# Copy scores
if os.path.exists('scores.json'):
    shutil.copy2('scores.json', os.path.join(output_dir, 'scores.json'))

# Copy fold models if they exist
training_dir = 'artifacts/training'
if os.path.isdir(training_dir):
    os.makedirs(os.path.join(output_dir, 'fold_models'), exist_ok=True)
    for f in os.listdir(training_dir):
        if f.startswith('model_fold_') and f.endswith('.h5'):
            shutil.copy2(
                os.path.join(training_dir, f),
                os.path.join(output_dir, 'fold_models', f)
            )

# Zip everything
shutil.make_archive('/kaggle/working/trained_outputs', 'zip', output_dir)
print('Output saved to /kaggle/working/trained_outputs.zip')
print('Download from the Output tab or use the Kaggle API.')

## 7) Auto-stop session after completion

The next cell runs automatically: it verifies outputs are saved, then stops the Kaggle kernel so you can close the browser without worrying about compute still running.

**What happens:**
1. Checks that `trained_outputs.zip`, `model.h5`, and `scores.json` exist.
2. If all present, shuts down the kernel automatically.
3. You can safely close your browser — Kaggle outputs are already saved.

In [ ]:
import os
import time

# Check that critical outputs exist
required_files = [
    '/kaggle/working/trained_outputs.zip',
    '/kaggle/working/Kidney-Disease-Classification-System/model/model.h5',
    '/kaggle/working/Kidney-Disease-Classification-System/scores.json'
]

missing = [f for f in required_files if not os.path.exists(f)]

if missing:
    print(f"⚠️  WARNING: {len(missing)} output file(s) not found. Not stopping kernel.")
    print("Missing files:")
    for f in missing:
        print(f"  - {f}")
    print("\nRun 'Download trained outputs' cell above to create them.")
else:
    print("✅ All outputs found and saved successfully!")
    print(f"  - trained_outputs.zip: {os.path.getsize(required_files[0]) / 1e6:.1f} MB")
    print(f"  - model.h5: {os.path.getsize(required_files[1]) / 1e6:.1f} MB")
    print(f"  - scores.json: EXISTS")
    print("\n🛑 Shutting down kernel in 5 seconds...")
    print("   You can now safely close your browser. Outputs are on Kaggle.")
    
    for countdown in range(5, 0, -1):
        print(f"   {countdown}...", flush=True)
        time.sleep(1)
    
    print("\n   Goodbye!")
    os._exit(0)